In [3]:
# Automation using a class

from math import isnan
import xlrd
import openpyxl 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

cats = ['rules','eng','arg','sup','str','tim','pol','ora']
pcats = [x+'a' for x in cats]
ncats = [x+'n' for x in cats]
numcat = dict(zip([x for x in range(8)],cats))
def zscore(li):
    mn = li.mean()
    st = li.std()
    return (li - mn) / st

class Analyze:
    def __init__(self,file):
        self.file = file

        self.workbook = openpyxl.load_workbook(self.file)
        w = self.workbook['ballots']
        j = self.workbook['judges']
        d = self.workbook['debaters']
        ji = self.workbook['judge info']
        first_row = list(next(w.values))
        self.jirow = list(next(ji.values))

        judgesname = {j.cell(i,2).value:i for i in range(1,1+j.max_row)}
        judgesnumber = {x:y for y,x in judgesname.items()}
        debatersname = {d.cell(i,2).value:i for i in range(1,1+d.max_row)}
        debatersnumber = {x:y for y,x in debatersname.items()}
        self.judgesname = judgesname
        self.judgesnumber = judgesnumber
        self.debatersname = debatersname
        self.debatersnumber = debatersnumber

        data = []
        for row in range(2, 1+w.max_row):
            elm = {}
            for col in range(1,1+w.max_column):
                elm[first_row[col-1]]=w.cell(row,col).value
            data.append(elm)

        ballots = pd.DataFrame(data)

        def rd(row):
            if row['Round'] == 'Round 1':
                val = 1
            elif row['Round'] == 'Round 2':
                val = 2
            else:
                val = 3
            return val

        win = {'Affirmative':1,'Negative':0}

        ballots['rd'] = ballots.apply(rd,axis=1)
        ballots['win'].replace(win,inplace=True)
        ballots['Aff'].replace(debatersname,inplace=True)
        ballots['Negative'].replace(debatersname,inplace=True)
        ballots['Judge'].replace(judgesname,inplace=True)
        ballots.rename(columns={'Negative':'Neg'},inplace=True)

        self.ballots = ballots

        cats = ['rules','eng','arg','sup','str','tim','pol','ora']
        numcat = dict(zip([x for x in range(8)],cats))
        pcats = [x+'a' for x in cats]
        ncats = [x+'n' for x in cats]
        aff, neg, allcat = pcats, ncats, cats
        aff.append('Aff')
        neg.append('Neg')
        allcat.append('deb')
        dicn = dict(zip(ncats,cats))
        dicp = dict(zip(pcats,cats))
        dicaff = dict(zip(aff,allcat))
        dicneg = dict(zip(neg,allcat))

        def debpts(i):
            one = ballots[(ballots['Aff']==i) | (ballots['Neg']==i)]
            n = one[one['Neg']==i][ncats]
            p = one[one['Aff']==i][pcats]
            n.rename(columns=dicn,inplace=True)
            p.rename(columns=dicp,inplace=True)
            tot = n.append(p)
            return tot

        def judbal(i):
            two = ballots[ballots['Judge']==i]
            b = two[neg]
            c = two[aff]
            b = b.rename(columns=dicneg)
            c = c.rename(columns=dicaff)
            tot = b.append(c)
            return tot

        dicdebarr = {x:debpts(x) for x in range(1,1+len(debatersname))}
        dicdebavg = {x:dicdebarr[x].mean() for x in range(1,1+len(debatersname))}
        dicdebstd = {x:dicdebarr[x].std() for x in range(1,1+len(debatersname))}
        self.avgdebs = pd.DataFrame(dicdebavg)
        self.dicdebarr = dicdebarr
        stddebs = pd.DataFrame(dicdebstd)

        dicjudarr = {x:judbal(x) for x in range(1,1+len(judgesname))}
        dicjudavg = {x:dicjudarr[x].mean() for x in range(1,1+len(judgesname))}
        dicjudstd = {x:dicjudarr[x].std() for x in range(1,1+len(judgesname))}
        avgjuds = pd.DataFrame(dicjudavg)
        stdjuds = pd.DataFrame(dicjudstd)

        def firstpass(j,d):
            jscore = dicjudarr[j][dicjudarr[j]['deb']==d]
            jcatscore = jscore / dicdebavg[d] 
            jcatscore = jcatscore.drop(columns=['deb'])
            jdavg = np.mean(jcatscore.values)
            return jdavg

        def secondpass(j):
            avgs = []
            for o in range(1,1+len(debatersname)):
                try:
                    avgs.append(firstpass(j,o))
                except:
                    pass
            new = [x for x in avgs if isnan(x) != True]
            return [np.mean([abs(1-x) for x in new]), np.std(new)]

        def thirdpass(jfqr={}):
            for j in range(1,1+len(judgesname)):
                jfqr[j] = secondpass(j)
            return jfqr

        jfqr = thirdpass()
        jfqrset = pd.DataFrame(jfqr).T
        jfqrset = jfqrset.rename(columns={0:'fqr',1:'std'})
        FQavg = jfqrset['fqr'].mean()
        FQstd = jfqrset['fqr'].std()
        FQSTDavg = jfqrset['std'].mean()
        FQSTDstd = jfqrset['std'].std()

        Jfqz = (jfqrset['fqr'] - FQavg) / FQstd
        Jstdz = (jfqrset['std'] - FQSTDavg) / FQSTDstd
        jfqrset['jfqz'] = Jfqz
        jfqrset['jstdz'] = Jstdz
        jfqrset['Jp'] = (jfqrset['fqr'] + jfqrset['std'])
        jfqrset['Jfdif'] = 1 - jfqrset['Jp']

        jl = pd.Series(judgesnumber)
        jfqrset['judge'] = pd.Series(jl).T
        fdif = jfqrset['Jfdif']
        judges_master = pd.DataFrame([jl,fdif]).T
        judges_master = judges_master.rename(columns={'Unnamed 0':'Judge'})
        fmr = [ji.cell(i,3).value for i in range(2,1+ji.max_row)]
        inst = [ji.cell(i,4).value for i in range(2,1+ji.max_row)]
        nat = [ji.cell(i,5).value for i in range(2,1+ji.max_row)]
        yrs_jud = [ji.cell(i,6).value for i in range(2,1+ji.max_row)]
        ysg = [ji.cell(i,7).value for i in range(2,1+ji.max_row)]
        wins = [ji.cell(i,8).value for i in range(2,1+ji.max_row)]
        tard = [ji.cell(i,9).value for i in range(2,1+ji.max_row)]
        mist = [ji.cell(i,10).value for i in range(2,1+ji.max_row)]
        judges_master['Jf z'] = zscore(judges_master['Jfdif'])
        judges_master['former debater'],judges_master['instructor'],judges_master['native speaker'],judges_master['yrs jud'],judges_master['ysg'],judges_master['wins'],judges_master['late'],judges_master['mistake'] = [fmr,inst,nat,yrs_jud,ysg,wins,tard,mist]
    
        self.master = judges_master
        self.fairness = jfqrset
    
        # An attempt at finding split ballots
        ## Making a room list
        byround = self.ballots.groupby('rd')
        hold = byround.get_group(2).sort_values(by='Aff')
        temp0 = hold[hold.groupby('Aff').Aff.transform('count')>1]
        temp1 = hold[hold.groupby('Aff').Aff.transform('count')==1]
        test0 = [list(temp0[(i*3):3+(i*3)]['Judge']) for i in range(0,(1+len(temp0))//3)]
        test1 = list(temp1['Judge'])
        room = {x:x for x in test1}
        for x in test0:
            for y in x:
                room[y] = np.prod(x)
        self.room = room

        ## Assigning rooms
        jsplit = {x:0 for x in self.judgesname.keys()}
        dwin = {x:0 for x in self.debatersname.keys()}
        self.ballots['room'] = [room[x] for x in self.ballots['Judge'].values]
        byroom = self.ballots.groupby('room')
        rms = [x for x in byroom.groups.keys()]

        for i in rms:
            for l in range(1,4):
                if byroom.get_group(i)[byroom.get_group(i)['rd']==l]['win'].mean() > .5:
                    twe = byroom.get_group(i)[byroom.get_group(i)['rd']==l]
                    namea = self.debatersnumber[int(twe['Aff'].mean())]
                    dwin[namea] += 1
                else:
                    twe = byroom.get_group(i)[byroom.get_group(i)['rd']==l]
                    namen = self.debatersnumber[int(twe['Neg'].mean())]
                    dwin[namen] += 1
            if len(byroom.get_group(i)) > 3:
                for k in range(1,4):
                    if 0 < byroom.get_group(i)[byroom.get_group(i)['rd']==k]['win'].mean() < .5:
                        wer = byroom.get_group(i)[byroom.get_group(i)['rd']==k]
                        name = self.judgesnumber[int(wer[wer['win']==1]['Judge'].values)]
                        jsplit[name] += 1
                    elif .5 < byroom.get_group(i)[byroom.get_group(i)['rd']==k]['win'].mean() < 1:
                        wer = byroom.get_group(i)[byroom.get_group(i)['rd']==k]
                        name = self.judgesnumber[int(wer[wer['win']==0]['Judge'].values)]
                        jsplit[name] += 1
        
        self.dwins = dwin
        self.master['split'] = jsplit.values()
        self.master['rating'] = ((2 * self.master['former debater']) + (1.5 * self.master['instructor']) + (1 * self.master['native speaker']) + (self.master['split']/6) + (self.master['wins']/6) + (4.5 * self.master['Jfdif']) - self.master['mistake'] + (2 * ((((self.master['yrs jud'] + self.master['ysg'])/2)-self.master['late'])/4))) * (5/12) 

    def add_sheet(self, what, name):
        writer = pd.ExcelWriter(self.file, engine = 'openpyxl')
        writer.book = self.workbook
        what.to_excel(writer, sheet_name = name)
        writer.save()
        writer.close()
        print('done')

In [6]:
# Initialization
fair2020 = Analyze('/content/ballots2020.xlsx')
fair2019 = Analyze('/content/ballots2019.xlsx')

In [7]:
# Aff vs Neg overall: >.5 Aff was stronger, <.5 Neg was stronger in tourney
side19 = sum(fair2019.ballots['win'])/len(fair2019.ballots['win'])
side20 = sum(fair2020.ballots['win'])/len(fair2020.ballots['win'])
print('2019: {:.3f}\n2020: {:.3f}'.format(side19,side20))

2019: 0.536
2020: 0.522


In [14]:
# For returning judges, this is a combinatory approach to rating
j19 = set(list(fair2019.judgesname.keys()))
j20 = set(list(fair2020.judgesname.keys()))
returners = j19.intersection(j20)

ret19 = fair2019.master[fair2019.master['Judge'].isin(list(returners))]
ret20 = fair2020.master[fair2020.master['Judge'].isin(list(returners))]
ret19=ret19[['Judge','Jfdif','rating']]
ret20=ret20[['Judge','Jfdif','rating']]
ret20=ret20.reset_index(drop=True)
ret19=ret19.reset_index(drop=True)

ret20[['Jfdif 19','rating 19']] = ret19[['Jfdif','rating']]
ret20['comb rating'] = (ret20['rating'] * .75) + (ret20['rating 19'] * .25)
ret20['comb rating'] = ret20['comb rating'].apply(lambda x: round(x,2))
ret20.sort_values(by='comb rating',ascending=False)

,Judge,Jfdif,rating,Jfdif 19,rating 19,comb rating
0,Zach Bullard,0.805855,4.21931,0.867626,4.33513,4.25
14,Харитонова Елена,0.808952,3.39178,0.876408,3.44882,3.41
12,Josh Wilson,0.848263,3.25716,0.769273,3.10905,3.22
6,Шусть Наталья Петровна,0.83093,3.15522,0.819439,2.99478,3.12
1,Назарова Ксения Валерьевна,0.736774,2.83978,0.89546,3.20677,2.93
10,Добролюбова Ольга Евгеньевна,0.736373,2.73487,0.776958,3.12346,2.83
16,Замалетдинов Даниил,0.769052,2.76142,0.825354,2.10309,2.60
15,Талинский Иван,0.873815,2.47174,0.714038,2.79716,2.55
3,Черникова Надежда Алексеевна,0.861457,2.51801,0.858379,2.33863,2.47
13,Ильевская Мария,0.631782,2.43459,0.818215,2.36749,2.42


In [21]:
ro = fair2020.master['rating']
mwoz, mwz = ro.drop(1).mean(), ro.mean()
# first number is the judges' avgerage rating without ZB, the second is with

In [36]:
fair2020.master[['Judge','Jfdif','Jf z']].sort_values(by='Jf z', ascending=False)

,Judge,Jfdif,Jf z
16,Аринцева Мария Александровна,0.880279,1.24916
22,Талинский Иван,0.873815,1.16368
25,Суслова Елизавета,0.868333,1.09118
4,Черникова Надежда Алексеевна,0.861457,1.00025
18,Josh Wilson,0.848263,0.825767
12,Mike Cole,0.847071,0.81
6,Гунина Мария Николаевна,0.845237,0.785743
7,Шусть Наталья Петровна,0.83093,0.596539
26,Christopher Korten,0.824582,0.512585
5,Мищенко Настасья Владимировна,0.822945,0.490938


In [32]:
pd.Series([(x-mwz)/ro.std() for x in ro])

0     3.046553
1     0.424851
2    -0.830422
3    -0.186660
4    -0.455867
5    -0.772357
6     1.024308
7    -1.177667
8          NaN
9    -1.152996
10   -0.622171
11   -0.171935
12         NaN
13   -1.369549
14    0.225459
15   -0.647491
16         NaN
17    1.218047
18   -0.345192
19    0.562149
20    1.473891
21   -0.274600
22    0.275919
23   -0.291948
24    0.299750
25   -0.252072
dtype: float64

In [35]:
fair2020.master['zsc rat'] = [(x-mwz)/ro.std() for x in ro]
fair2020.master[['Judge','rating', 'zsc rat']].sort_values(by='zsc rat',ascending=False)

,Judge,rating,zsc rat
1,Zach Bullard,4.21931,3.046553
21,Харитонова Елена,3.39178,1.473891
18,Josh Wilson,3.25716,1.218047
7,Шусть Наталья Петровна,3.15522,1.024308
20,Земцовская Анна,2.91203,0.562149
2,Назарова Ксения Валерьевна,2.83978,0.424851
25,Суслова Елизавета,2.77396,0.299750
23,Замалетдинов Даниил,2.76142,0.275919
15,Добролюбова Ольга Евгеньевна,2.73487,0.225459
12,Mike Cole,2.52576,-0.171935


In [ ]:
fair2020.ballots

,Round,Judge,Aff,rulesa,enga,arga,supa,stra,tima,pola,oraa,Neg,rulesn,engn,argn,supn,strn,timn,poln,oran,win,ID,JRIDA,JRIDN,rd,room
0,Round 1,1,3,9,7,14,12,4,3,5,8,15,7,7,12,14,4,2,5,6,1,101,671,6710,1,360
1,Round 1,15,3,8,7,12,10,4,3,5,6,15,5,7,10,7,3,3,4,7,1,1501,10051,100510,1,360
2,Round 1,3,11,10,8,17,10,5,5,5,5,7,7,7,15,10,2,3,5,5,1,301,2015,20114,1,1248
3,Round 1,26,11,7,6,15,11,4,4,5,4,7,6,4,13,9,2,2,5,2,1,2601,17425,174214,1,1248
4,Round 1,23,4,7,4,14,9,4,4,3,6,13,3,4,8,10,3,3,5,4,1,2301,15419,154118,1,2530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Round 3,1,14,9,10,15,10,4,4,5,10,8,9,7,13,12,4,4,5,10,1,103,6716,676,3,360
65,Round 3,15,14,10,10,18,15,5,4,5,10,8,10,7,15,12,5,5,5,8,1,1503,100516,10056,3,360
66,Round 3,2,18,10,9,18,12,5,5,4,8,4,10,8,17,12,5,5,5,9,1,203,1348,1349,3,2
67,Round 3,24,14,10,10,14,15,4,5,5,10,8,9,8,18,11,5,5,3,9,1,2403,160816,16086,3,360


In [9]:
avgs = fair2020.avgdebs.T
avgs = avgs[avgs.columns[0:len(avgs.columns)-1]]
avgs['sum'] = avgs.sum(axis=1)
avgs

,rules,eng,arg,sup,str,tim,pol,ora,sum
1,8.222222,5.555556,13.111111,8.000000,3.555556,4.555556,4.777778,5.555556,53.333333
2,8.428571,8.714286,12.285714,9.000000,4.142857,3.428571,5.000000,7.428571,58.428571
3,9.111111,6.777778,13.777778,11.444444,4.000000,3.444444,4.666667,7.000000,60.222222
4,8.857143,4.857143,12.142857,9.285714,3.857143,4.428571,3.428571,5.857143,52.714286
5,6.000000,7.000000,10.800000,9.000000,2.800000,3.200000,4.600000,6.200000,49.600000
6,8.857143,6.000000,11.571429,11.571429,3.285714,4.428571,4.142857,4.857143,54.714286
7,4.777778,5.222222,8.777778,10.111111,2.222222,2.666667,4.555556,3.666667,42.000000
8,9.428571,7.000000,14.428571,10.000000,4.285714,4.428571,4.714286,8.000000,62.285714
9,8.800000,8.000000,16.000000,13.000000,4.200000,3.800000,3.800000,7.600000,65.200000
10,7.142857,7.142857,12.857143,8.428571,4.142857,3.857143,4.857143,5.857143,54.285714


In [10]:
# votes
def votes(year):
    votes = {x:0 for x in year.debatersname.keys()}
    sub = year.ballots[['Aff','Neg','win']]
    for i in range(len(sub)):
        sl = sub[i:(i+1)]
        if int(sl['win'])==1:
            n = year.debatersnumber[int(sl['Aff'])]
        else:
            n = year.debatersnumber[int(sl['Neg'])]
        votes[n] += 1
    return pd.Series(votes)

votes20 = votes(fair2020)
votes19 = votes(fair2019)
votes19.sort_values()

# calculate wins
win20 = pd.Series(fair2020.dwins)
win19 = pd.Series(fair2019.dwins)

# calculate avgs - unfinished!
def avcal(year,deb):
    return year.dicdebarr[deb].drop(columns='deb').mean()

av20 = pd.DataFrame({fair2020.debatersnumber[x]:avcal(fair2020,x) for x in fair2020.debatersnumber.keys()}).T
av19 = pd.DataFrame({fair2019.debatersnumber[x]:avcal(fair2019,x) for x in fair2019.debatersnumber.keys()}).T

# master table of debaters
debsmaster20 = pd.DataFrame([win20,votes20]).T
debsmaster20 = debsmaster20.rename(columns={0:'wins',1:'votes'})

debsmaster19 = pd.DataFrame([win19,votes19]).T
debsmaster19 = debsmaster19.rename(columns={0:'wins',1:'votes'})

debsmaster20[cats] = av20
debsmaster19[cats] = av19

debsmaster19 = debsmaster19.reset_index().rename(columns={'index':'debater'})
debsmaster20 = debsmaster20.reset_index().rename(columns={'index':'debater'})

In [11]:
# Framework for analyzing returning debaters
de19 = set(fair2019.debatersname.keys())
de20 = set(fair2020.debatersname.keys())
both = list(de19.intersection(de20))
both

# started....
def changed(deb):
    new = debsmaster20[debsmaster20['debater']==both[deb]]
    new = new.rename({int(new.index.values):'new'})
    old = debsmaster19[debsmaster19['debater']==both[deb]]
    old = old.rename({int(old.index.values):'old'})
    di = pd.DataFrame(100 * (new[cats].values - old[cats].values) / new[cats].values).rename(columns=numcat,index={0:'% dif'})
    return new.append(old).append(di)

bothdf = pd.DataFrame()
for x in range(len(both)):
    bothdf = bothdf.append(changed(x))
bothdf

,debater,wins,votes,rules,eng,arg,sup,str,tim,pol,ora
new,Тулупова Алиса,3.0,6.0,9.571429,7.857143,16.428571,13.142857,4.571429,4.714286,4.857143,7.142857
old,Тулупова Алиса,3.0,8.0,9.555556,7.111111,16.000000,11.444444,4.777778,4.222222,4.666667,7.222222
% dif,NaN,NaN,NaN,0.165837,9.494949,2.608696,12.922705,-4.513889,10.437710,3.921569,-1.111111
new,Бордачев Олег,2.0,5.0,8.857143,4.857143,12.142857,9.285714,3.857143,4.428571,3.428571,5.857143
old,Бордачев Олег,2.0,6.0,9.000000,7.555556,14.000000,11.444444,4.777778,4.555556,4.555556,7.888889
% dif,NaN,NaN,NaN,-1.612903,-55.555556,-15.294118,-23.247863,-23.868313,-2.867384,-32.870370,-34.688347
new,Зозуля Елена,2.0,3.0,9.428571,7.000000,14.428571,10.000000,4.285714,4.428571,4.714286,8.000000
old,Зозуля Елена,2.0,5.0,6.888889,4.444444,9.000000,6.333333,2.888889,3.777778,4.333333,4.888889
% dif,NaN,NaN,NaN,26.936027,36.507937,37.623762,36.666667,32.592593,14.695341,8.080808,38.888889
new,Соловьева Юлия,2.0,5.0,9.285714,8.857143,16.714286,11.571429,4.714286,4.000000,4.857143,8.428571


In [12]:
fair2020.ballots[(fair2020.ballots['Aff']==fair2020.debatersname['Игнатова Ксения'])|(fair2020.ballots['Neg']==fair2020.debatersname['Игнатова Ксения'])][['Judge','Aff','Neg','win']]

,Judge,Aff,Neg,win
5,18,9,16,1
27,2,2,9,1
46,26,1,9,0
47,16,1,9,0
48,3,1,9,0


In [18]:
# fair2019.add_sheet(fair2019.master,'rating')
# fair2020.add_sheet(fair2020.master,'rating')
# fair2020.add_sheet(ret20,'returning judges')
# fair2020.add_sheet(bothdf,'returning debs')

# Новый раздел